# Lab 4 Activity

In [1]:
import pandas as pd
import numpy as np
from random import randrange
from IPython.display import Image
np.set_printoptions(precision = 3)

## Load Data

In [2]:
data_pd = pd.read_csv("jester-data-1.csv", index_col=False, usecols=range(1, 101))
d = data_pd.to_latex()
text_file = open("Output_jester.txt", "w")
text_file.write(d)
text_file.close()

## Pre-processing

In [9]:
data = data_pd.values
train_set = data.copy()
val_examples = []

for i in range(int(data.size*0.1)):
    x = randrange(data.shape[0])
    y = randrange(data.shape[1])
    while data[x, y] == 99 or (x, y) in val_examples:
        x = randrange(data.shape[0])
        y = randrange(data.shape[1])
    val_examples.append((x, y))
    train_set[x, y] = 99

KeyboardInterrupt: 

## Initialize Preferences and Features

In [4]:
n_features = 2

latent_user_preferences = np.random.random((train_set.shape[0], n_features))
latent_item_features = np.random.random((train_set.shape[1],n_features))

## Recommender System

In [5]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(ratings, iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = ratings[user_id][item_id]
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if(iteration%1000 == 0 ):
            print(mse)


## Train

In [6]:
sgd(train_set, iterations=10000)

26.141555425182396


KeyboardInterrupt: 

## Perform Predictions

In [7]:
predictions = latent_user_preferences.dot(latent_item_features.T)

## Performance Measures

In [8]:
error = []

for (x, y) in val_examples:
    err = data[x, y] - predictions[x, y]
    error.append(err)
    
mse = (np.array(error) ** 2).mean()

print("Algorithm MSE:", mse)

Algorithm MSE: 22.64150238634639
